In [0]:
import numpy as np 
import pandas as pd 

import pandas_datareader.data as web

import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from datetime import datetime
import time
import ipywidgets as wg
#from IPython.display import display
%matplotlib inline

In [0]:
start = '2010-04'
end = '2020-03'

ff_3f  = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0]

ff_48  = web.DataReader('48_Industry_Portfolios','famafrench', start, end)[0]

data = pd.concat([ff_3f, ff_48], axis=1, sort=False)

In [0]:
for i in range(4,len(data.columns)):
    data[data.columns[i]] = data[data.columns[i]] - data['RF']
new_data =data.drop(columns=['SMB','HML','RF'])
e_r = np.mean(new_data.iloc[:,1:])
x = sm.add_constant(new_data['Mkt-RF'])

### Modo1

In [0]:
window = 24 # rolling window

def rolling(y: 'new_data', x: 'Mkt-Rf', t:'start of rolling') -> 'plot':
    betas = []
    index = {}
    for i in range(1, len(y.columns)):
        reg = sm.OLS(y.iloc[t:t+window, i], x[t:t+window]).fit()
        betas.append(reg.params[1])
    betas = pd.Series(betas, e_r.index)
    
    for i in range(len(new_data.index)-window):
        index[i] = 'Da ' + str(new_data.index[i]) + ' a ' + str(new_data.index[i+window]) # Per il titolo

    
    plt.figure(figsize=(15,8))
    ax = sns.regplot(betas,e_r)
    for i, txt in enumerate(e_r.index):
        ax.annotate(txt, (betas[i], e_r[i]))
    plt.ylabel('Expected Return')
    plt.xlabel('Beta')
    plt.title(index[t])

c_slide = wg.FloatSlider(value=0, min=0, max=len(new_data.index)-window-2, step=1)
wg.interact(lambda t: rolling(new_data, x, int(t)), t = c_slide)
#
#for i in range(len(new_data.index)-window-2):
#    c_slide.value = i
#    time.sleep(0.05)

interactive(children=(FloatSlider(value=0.0, description='t', max=94.0, step=1.0), Output()), _dom_classes=('w…

<function __main__.<lambda>>

### Modo 2

In [0]:
#beta = {}
#index = {}
#window = 24 # rolling window
#for i in range(len(new_data.index)-window-1):
#    index[i] = 'Da ' + str(new_data.index[i]) + ' a ' + str(new_data.index[i+window]) # Per il titolo
#    beta[i] = []
#    for j in range(1, len(new_data.columns)):
#        beta[i].append(sm.OLS(new_data.iloc[i:i+window,j], x.iloc[i:i+window]).fit().params[1])
#        
#def graph(t):
#    '''I dati sono mensili, per cui 10 anni sono 120 mesi: la finestra rolling è 2 anni, quindi 24 mesi.
#    Di conseguenza, ci saranno 120 - 24 = 96 regressioni rolling
#    t indica quale tra queste regressioni viene scelta, dove t = 0 è la prima
#    '''
#    plt.figure(figsize=(15,8))
#    ax = sns.regplot(beta[t],e_r)
#    for i, txt in enumerate(e_r.index):
#        ax.annotate(txt, (beta[t][i], e_r[i]))
#    plt.ylabel('Expected Return')
#    plt.xlabel('Beta')
#    plt.title(index[t])
#    
#c_slide = wg.FloatSlider(value=0, min=0, max=len(new_data.index)-window-1, step=1)
#wg.interact(graph, t = c_slide)